In [ ]:
import urllib
from bs4 import BeautifulSoup	#采用BeautifulSoup
import os
import re

#伪装成浏览器登陆,获取网页源代码
def getPage(href): 
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'
    }
    req = urllib.request.Request(
        url = href ,
        headers = headers
    )
    try:
        post = urllib.request.urlopen(req)
    except urllib.error.HTTPError as e:
        print(e.code)
        print(e.reason)
    return post.read()

#初始化url 双色球首页
url = 'http://kaijiang.zhcw.com/zhcw/html/ssq/list_1.html'  


#===============================================================================
#获取url总页数
def getPageNum(url):
    num =0
    page = getPage(url)
    soup = BeautifulSoup(page)	
    strong = soup.find('td',colspan='7')
    # print strong
    if strong:
        result = strong.get_text().split(' ')
        # print result
        list_num = re.findall("[0-9]{1}",result[1])
        # print list_num
        for i in range(len(list_num)):
            num = num*10 + int(list_num[i])
        return num
    else:
        return 0

#===============================================================================
#获取每页双色球的信息
def getText(url):

    for list_num in range(1,117):	#从第一页到第getPageNum(url)页
        print(list_num)   #打印下页码
        href = 'http://kaijiang.zhcw.com/zhcw/html/ssq/list_'+str(list_num)+'.html' #调用新url链接
        # for listnum in len(list_num):
        page = BeautifulSoup(getPage(href))
        em_list = page.find_all('em')	#匹配em内容
        div_list = page.find_all('td',{'align':'center'})	#匹配 <td align=center>这样的内容

		#初始化n
        n = 0
		#将双色球数字信息写入num.txt文件
        fp = open("num.txt" ,"w")
        for div in em_list:
            emnum1 = div.get_text()
            # print emnum1
            text = div.get_text()
            #text = text.encode('utf-8') #remove by shuaifei for windows
            #print title       
            n=n+1
            if n==7:
                text = text + "\n"
                n=0
            else:
                text = text + ","
            fp.write(str(text))
        fp.close()

		#将日期信息写入date.txt文件
        fp = open("date.txt" ,"w")
        for div in div_list:
            text = div.get_text().strip('')
            #print(text)
            list_num = re.findall('\d{4}-\d{2}-\d{2}',text)
            list_num = str(list_num[::1])
            list_num = list_num[2:12]
            #print(list_num)
            if len(list_num) == 0:
                continue
            elif len(list_num) > 1:
                fp.write(str(list_num)+'\n')
        fp.close()
		
		#将num.txt和date.txt文件进行整合写入hun.txt文件中
		#格式如下：
		#('2016-05-03', '09,12,24,28,29,30,02')
		#('2016-05-01', '06,08,13,14,22,27,10')
		#('2016-04-28', '03,08,13,14,15,30,04')
		#
        fp01 = open("date.txt","r")
        a=[]
        for line01 in fp01:
            a.append(line01.strip('\n'))
            # print a
        fp01.close()

        fp02 = open("num.txt","r")
        b=[]
        for line02 in fp02:
            b.append(line02.strip('\n'))
            # print b

        fp02.close()

        fp = open("hun.txt" ,"a")
        for cc in zip(a,b):	#使用zip方法合并
            print(cc)
            fp.write(str(cc) + '\n')
        fp.close()

pageNum = getPageNum(url)	
print(pageNum)
getpagetext = getText(url)  
print(getpagetext)

In [ ]:
import pickle
import sys
import getopt
import pandas as pd
import numpy as np
from collections import Counter
from collections import defaultdict
from collections import OrderedDict

class ssqfx:
    def __init__(self):
        self._ssqmap=OrderedDict()    #{}
        self._allmap=OrderedDict()    #{}
    def add_ssq(self,date,ssq):
        self._ssqmap[date]=ssq
    def add_allssq(self,datestr,allnum):
        self._allmap[datestr]=allnum
    def __iter__(self):
        return self
    def get_map(self):
        getmap=self._ssqmap
        return getmap
    def __next__(self):
	    return self._ssqmap.__next__()
    @staticmethod
    def get_union_len(lis1,lis2):
	    listmp=list(set(lis1).intersection(set(lis2)))
	    return len(listmp)
    def get_union(self,date,num,unimap):
	    if not self._ssqmap[date]:
	        unimap={}
	    for item in self._ssqmap:
	        if self.get_union_len(self._ssqmap[date],self._ssqmap[item])==num:
		        unimap[item]=self._ssqmap[item]
    def get_year_str(self,start,ylis):
        for item in self._ssqmap:
            if item.startswith(start):
                ylis.append(item)
    def get_value(self,date):
        return self._ssqmap[date]
    
    def get_sum_cnt(self,year='2019'):
        sumlis=[]
        yearlis=[]*160
        pos=0
        if year == '2019':
            for i in self._ssqmap:
                sumlis.append(sum_lis(self._ssqmap[i]))
                pos += 1
        else:
            self.get_year_str(year,yearlis)
            for i in yearlis:
                sumlis.append(sum_lis(self._ssqmap[i]))
                pos += 1
        return sumlis

    def get_digt_cnt(self,year='2019'):
        digtdict=defaultdict(lambda: 0)
        if year == '2019':
            for i in self._ssqmap:
                for j in self._ssqmap[i]:
                    digtdict[j] += 1
        else:
            yearl=[i for i in self._ssqmap if i.startswith(year)]
            for j in yearl:
                for k in self._ssqmap[j]:
                    digtdict[k] += 1
        return digtdict
    def add_by_date(self,start,end):
        ssqdate=[v for i,v in self._ssqmap.items() if cmp(i,start) >= 0 and cmp(end,i) >= 0]
        """ssqdate=[]*2000
        for i in self._ssqmap:
            j=int(i.replace('-',''))
            if j >= start and j <= end:
                ssqdate.append(i)"""
        #ssqdate=[i for i in self._ssqmap if int(i.replace('-','')) >= start and int(i.replace('-','')) <= end]
        return ssqdate
    def get_date_pos(lis,pos):
        return lis[pos]
    def create_xsl(self):
        dictdf=OrderedDict()
        r1=[]*len(self._ssqmap)
        r2=[]*len(self._ssqmap)
        r3=[]*len(self._ssqmap)
        r4=[]*len(self._ssqmap)
        r5=[]*len(self._ssqmap)
        r6=[]*len(self._ssqmap)
        b1=[]*len(self._ssqmap)
        datalist=[i for i in self._allmap]
        datalist=sorted(datalist)
        for d in datalist:
            j=self._allmap[d]
            if len(j)==7:
                r1.append(j[0])
                r2.append(j[1])
                r3.append(j[2])
                r4.append(j[3])
                r5.append(j[4])
                r6.append(j[5])
                b1.append(j[6])
        dictdf['date']=datalist
        dictdf['r1']=r1
        dictdf['r2']=r2
        dictdf['r3']=r3
        dictdf['r4']=r4
        dictdf['r5']=r5
        dictdf['r6']=r6
        dictdf['b1']=b1
        df=pd.DataFrame(dictdf)
        #print df.sort_values(by=['date'])
        #print df
        df.to_excel("ssq.xls")
    def create_h5(self):
        dictdf=OrderedDict()
        r1=[]*len(self._ssqmap)
        r2=[]*len(self._ssqmap)
        r3=[]*len(self._ssqmap)
        r4=[]*len(self._ssqmap)
        r5=[]*len(self._ssqmap)
        r6=[]*len(self._ssqmap)
        b1=[]*len(self._ssqmap)
        datalist=[i for i in self._allmap]
        list2d=[r1,r2,r3,r4,r5,r6,b1]
        datalist=sorted(datalist)
        for j in datalist:
            for var,lis in zip(self._allmap[j],list2d):
                lis.append(var)
        dictdf['date']=datalist
        dictdf['r1']=r1
        dictdf['r2']=r2
        dictdf['r3']=r3
        dictdf['r4']=r4
        dictdf['r5']=r5
        dictdf['r6']=r6
        dictdf['b1']=b1
        #print("%d %d %d"%(len(datalist),len(r1),len(b1)))
        df=pd.DataFrame(dictdf)
        #print df.sort_values(by=['date'])
        df.to_excel("ssq.xls")
def sum_lis(lis):
    lsum=0
    for i in lis:
        lsum+=int(i)
    return lsum

def parse_line(sso,filenm):
    with open(filenm,'r') as f:
        for line in f:
            if line.strip().startswith('('):
                date,ssq=line.strip().split(' ')
                datestr=date[2:12]
                ssqstr=ssq[1:18]
                #print(datestr,ssqstr)
                datetup=tuple(datestr.split('-'))
                ssqnum=list(ssqstr.split(','))
		#print(datetup)
		#print(ssqnum)
                allstr=ssq[1:21]
                #print(allstr)
                #print('\n')
                allnum=list(allstr.split(','))
                sso.add_ssq(datestr,ssqnum)
                sso.add_allssq(datestr,allnum)

def usage():
    print("must have two args at least")
    print(sys.argv[0]+' num')

if __name__ == '__main__':
    cnum=4
    year='2018'
    last_year='2017'
    cntyear='2018'
    opts={}
    if len(sys.argv)<2:
        usage()
        sys.exit(-1)
    try:
        opts,args=getopt.getopt(sys.argv[1:],"c:y:a:")
    except:
        usage()
        #print(e)
    for opt,arg in opts:
        if opt == '-c':
	        cnum=int(arg)
        elif opt == '-y':
            year=arg
            last_year=str(int(year)-1)
        elif opt == '-a':
            cntyear=arg
    lis=[]*160
    sumlis=[]*160
    yearlist=[]*160
    sso=ssqfx()
    parse_line(sso,"hun.txt")
    #parse_line(sso,"myssq.txt")
    sso.get_year_str(year,lis)
    #print(lis)
    sso.create_h5()
    lis.sort()
    for i in lis:
        getmap={}
        summap={}
        sso.get_union(i,cnum,getmap)
        print("get the date of %s %s sum=%d"%(i,sso.get_value(i),sum_lis(sso.get_value(i))))
        sumlis.append(sum_lis(sso.get_value(i)))
        for item in getmap:
            print(item,getmap[item],sum_lis(getmap[item]))
	    #summap[sum_lis(getmap[item])] = 1 +summap[sum_lis(getmap[item])]
	    #sumlis.append(sum_lis(getmap[item]))
            if item.startswith(last_year):
                yearlist.append(item)
    m=Counter(sumlis)
    print(m)

    print('=============================')
    print(Counter(yearlist))
    print(sorted(yearlist))

    print('################################')
    print(Counter(sso.get_sum_cnt(cntyear)))
    print('********************************')
    print(sso.get_digt_cnt(cntyear))

    #print(sorted(sso.add_by_date("2017-11-01","2018-03-20")))
    #print(sorted(sso.add_by_date(20171101,20180320)))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import sys
import getopt
from collections import OrderedDict
import operator

class cgraph:
    def create_bar_sum(self,sumdic,gtitle='2018-01-01'):
        date_axis=[i for i in sumdic]
        lab_axis=np.arange(len(date_axis))
        sum_axis=[sum_lis(v) for i,v in sumdic.items()]
        print('mean of guess is %.3f'%(sum_lis(sum_axis)/len(sum_axis),))
        lab_axisy=np.arange(30,180,10)
        plt.figure()
        plt.title(gtitle)
        plt.xlabel('sum')
        plt.ylabel('date')
        plt.grid()
        plt.xticks(lab_axis,date_axis,rotation=90)
        plt.yticks(lab_axisy)
        plt.plot(lab_axis,sum_axis)
        #plt.show()

    def create_num_line(self,numdic,gtitle='2018-01-01'):
        lnum=defaultdict(lambda: 0)
        y_values=[]*34
        for _,v in numdic.items():
            for i in v:
                lnum[i] += 1
        #print lnum
        keys=sorted(lnum)
        #print lnum
        print(keys)
        keysarr=['01','02','03','04','05','06','07','08','09','10',
                  '11','12','13','14','15','16','17','18','19','20',
                  '21','22','23','24','25','26','27','28','29','30',
                  '31','32','33']
        for i in keysarr:
            y_values.append(lnum[i])
        lab_axisx=np.arange(1,34)
        lab_axisy=np.arange(100)
        plt.figure()
        plt.title(gtitle)
        plt.xlabel('num')
        plt.ylabel('count')
        plt.xticks(lab_axisx)
        plt.yticks(lab_axisy)
        plt.grid()
        plt.plot(lab_axisx,y_values)
        #plt.show()
        
class tjssq:
    def __init__(self,ssqdata):
        #self.sumarr=defaultdict(lambda: 0)
        self.sumarr=OrderedDict()
        self.num=defaultdict(lambda: 0)
        self._ssqmap=ssqdata

    def get_by_date(self,start,end):
        print(start)
        print(end)
        ssqdate=[i for i in self._ssqmap if operator.ge(i,start) and operator.ge(end,i)]
        ssqdate=sorted(ssqdate)
        for i in ssqdate:
            self.sumarr[i]=self._ssqmap[i]
        return self.sumarr
    def get_by_month(self,year,mon):
        month=[["2018-01-01","2018-01-31"],["2018-02-01","2018-02-31"],["2018-03-01","2018-03-31"],["2018-04-01","2018-04-31"],["2018-05-01","2018-05-31"],["2018-06-01","2018-06-31"],["2018-07-01","2018-07-31"],["2018-08-01","2018-08-31"],["2018-09-01","2018-09-31"],["2018-10-01","2018-10-31"],["2018-11-01","2018-11-31"],["2018-12-01","2018-12-31"]]
        start,end=month[int(mon)-1]
        start=start.replace('2018',year)
        end=end.replace('2018',year)
        return start,end
    def get_by_year(self,year):
        year=["2018-01-01","2018-12-31"]
        start,end=month[int(mon)-1]
        start=start.replace('2018',year)
        end=end.replace('2018',year)
        return start,end
        
def usage():
    print("this is alayis ssq")
    print("usage:")
    print("sys.argv[0] -y year -m month -c starttime:endtime -s selectmode")
        
if __name__ == '__main__':
    year="2018"
    month="01"
    select=''
    mtime=""
    if len(sys.argv) < 2:
        usage()
        sys.exit(-1)

    try:
        opts,args=getopt.getopt(sys.argv[1:],"y:m:c:s:")
    except:
        usage()
        #print(e)
    
    for opt,arg in opts:
        if opt == '-y':
            year=arg
        elif opt == '-m':
            month=arg
        elif opt == '-c':
            mtime=arg
        elif opt == '-s':
            select=arg
        else:
            print("no invalid arg")
    
    sso=ssqfx()
    parse_line(sso,"hun.txt")
    #datasc=tjssq(sso.get_map())
    #cg=cgraph()
    
    if ":" in mtime:
        start,end=mtime.split(':')
        datasc=tjssq(sso.get_map())
        cg=cgraph()
        cg.create_bar_sum(datasc.get_by_date(start,end),gtitle=start)
        cg.create_num_line(datasc.get_by_date(start,end),gtitle=start)
    elif select:
        for i in range(2):
            for j in range(6):
                if select == '2018':
                    if i==1 and j==1:
                        break
                datasc=tjssq(sso.get_map())
                cg=cgraph()
                tistr,_=datasc.get_by_month(select,i*6+j+1)
                cg.create_bar_sum(datasc.get_by_date(*datasc.get_by_month(select,i*6+j+1)),gtitle=tistr)
                cg.create_num_line(datasc.get_by_date(*datasc.get_by_month(select,i*6+j+1)),gtitle=tistr)
    else:    
        datasc=tjssq(sso.get_map())
        cg=cgraph()
        cg.create_bar_sum(datasc.get_by_date("2018-01-01","2018-06-01"))
        cg.create_num_line(datasc.get_by_date("2018-01-01","2018-06-01"))
    plt.show()


In [ ]:
import xlrd  
import xlwt  
from datetime import date,datetime
from xlutils.copy import copy

workbook = xlrd.open_workbook(r'ssq.xls')
print(workbook.sheet_names())
#sheet2 = workbook.sheet_names()[0]  
# 根据sheet索引或者名称获取sheet内容  
#sheet2 = workbook.sheet_by_index(1) # sheet索引从0开始  
sheet2 = workbook.sheet_by_name('Sheet1')  
print(sheet2.name,sheet2.nrows,sheet2.ncols)
rows = sheet2.row_values(4) # 获取第四行内容  
cols = sheet2.col_values(2) # 获取第三列内容  
print(rows)
#print(cols)
print(sheet2.cell(1,4).value.encode('utf-8'))
print(sheet2.cell_value(1,4).encode('utf-8'))
print(sheet2.row(1)[4].value.encode('utf-8'))  
# 获取单元格内容的数据类型  
print(sheet2.cell(1,0).ctype)

workbook2=copy(workbook)
sheet2w = workbook2.get_sheet(0)
pattern = xlwt.Pattern() # Create the Pattern
pattern.pattern = xlwt.Pattern.SOLID_PATTERN # May be: NO_PATTERN, SOLID_PATTERN, or 0x00 through 0x12
pattern.pattern_fore_colour = 5 # May be: 8 through 63. 0 = Black, 1 = White, 2 = Red, 3 = Green, 4 = Blue, 5 = Yellow, 6 = Magenta, 7 = Cyan, 16 = Maroon, 17 = Dark Green, 18 = Dark Blue, 19 = Dark Yellow , almost brown), 20 = Dark Magenta, 21 = Teal, 22 = Light Gray, 23 = Dark Gray, the list goes on...
style = xlwt.XFStyle() # Create the Pattern
style.pattern = pattern # Add Pattern to Style
sheet2w.write(2, 4,sheet2.cell(2,4).value, style=style)
#workbook2.save(r'ssq.xls')


In [ ]:
import xlrd  
import xlwt  
from datetime import date,datetime
from xlutils.copy import copy

def readexecl(excel,isheet):
    workbook=xlrd.open_workbook(excel)
    sheetr=workbook.sheet_by_index(isheet)
    return sheetr

def readcell(xpos,ypos,isheetr):
    return isheetr.cell(xpos,ypos)

def rowsrd(rows,isheetr):
    return isheetr.row_values(rows)
    
def getssq(isheetr,xs,xe):
    for i in range(isheetr.nrows):
        yield rowsrd(i,isheetr)[xs:xe]
    
def chmodopen(excel,isheet):
    workbook=xlrd.open_workbook(excel)
    workchmd=copy(workbook)
    return workchmd.getsheet(isheet),workchmd

def chmodecell(xpos,ypos,isheetc):
    pattern = xlwt.Pattern() # Create the Pattern
    pattern.pattern = xlwt.Pattern.SOLID_PATTERN # May be: NO_PATTERN, SOLID_PATTERN, or 0x00 through 0x12
    pattern.pattern_fore_colour = 2 # May be: 8 through 63. 0 = Black, 1 = White, 2 = Red, 3 = Green, 4 = Blue, 5 = Yellow, 6 = Magenta, 7 = Cyan, 16 = Maroon, 17 = Dark Green, 18 = Dark Blue, 19 = Dark Yellow , almost brown), 20 = Dark Magenta, 21 = Teal, 22 = Light Gray, 23 = Dark Gray, the list goes on...
    style = xlwt.XFStyle() # Create the style
    style.pattern = pattern # Add Pattern to Style
    isheetc.write(xpos,ypos,content,style)
    
def chctd(xpos,ypos,content,isheetc):
    isheetc.write(xpos,ypos,content)
    
def wtexll(isheetc,coll,rows):
    for i,j in enumerate(coll):
        chctd(rows,i,j,isheetc)
    
def chmodsave(workbook):
    workchmd.save(excel)
    
rsht=readexecl(r'ssq.xls',0)
print(rowsrd(2,rsht))
print(rowsrd(3,rsht)[2:8])
print(rsht.cell_value(2,5))

In [ ]:
from itertools import combinations
from collections import OrderedDict
import pandas as pd
import os
def crtexl(dlist,dtype):
    if dtype.startswith('comb6'):
        if len(dlist[0]) != 6:
            return 0
        else:
            pass
    elif dtype.startswith('sum_data'):
        dictdf=OrderedDict()
        #date=[]*len(dlist)
        r1=[]*len(dlist)
        r2=[]*len(dlist)
        r3=[]*len(dlist)
        r4=[]*len(dlist)
        r5=[]*len(dlist)
        r6=[]*len(dlist)
        #rsum=[]*len(dlist)
        #list2d=[r1,r2,r3,r4,r5,r6,rsum]
        for lis in dlist:
            r1.append(lis[0])
            r2.append(lis[1])
            r3.append(lis[2])
            r4.append(lis[3])
            r5.append(lis[4])
            r6.append(lis[5])
        #dictdf['date']=date
        dictdf['r1']=r1
        dictdf['r2']=r2
        dictdf['r3']=r3
        dictdf['r4']=r4
        dictdf['r5']=r5
        dictdf['r6']=r6
        #dictdf['sum']=rsum
        df=pd.DataFrame(dictdf) 
        dtype=dtype.replace('txt','xlsx')
        print(dtype)
        df.to_excel(dtype)

def cvt2xsl(fname):
    rlis=[]
    with open(fname,'rt') as f:
        for line in f:
            lis=list(line.split(' '))
            if len(lis) != 6:
                continue
            else:
                rlis.append(lis)
    crtexl(rlis,fname)

def findaf(dir):
    flis=os.listdir(dir)
    for i in flis:
        fp=os.path.join(dir,i)
        cvt2xsl(fp)
        
findaf('sum_data')

In [ ]:
import re
def sel_file(dir,sum):
    fils=os.listdir(dir)
    for i in fils:
        numf=re.findall(r"\d+",str(i))
        if numf and sum==numf[0] and i.endswith('xlsx'):
            return i

#print(sel_file('sum_data','90'))
